In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import layers
import keras

df = pd.read_csv("data/processed_games_with_eval.csv")
df = df.drop_duplicates(subset=["current_fen"])

2024-11-18 15:41:38.438560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-18 15:41:38.438616: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
def bitboard_to_np_array(bitboard: str) -> np.ndarray:
    return np.array([int(bit) for bit in bitboard], dtype=np.uint8)


df["p"] = df["p"].apply(bitboard_to_np_array)
df["r"] = df["r"].apply(bitboard_to_np_array)
df["n"] = df["n"].apply(bitboard_to_np_array)
df["b"] = df["b"].apply(bitboard_to_np_array)
df["q"] = df["q"].apply(bitboard_to_np_array)
df["k"] = df["k"].apply(bitboard_to_np_array)

In [ ]:
X = np.array(
    [
        np.stack([row["p"], row["r"], row["n"], row["b"], row["q"], row["k"]])
        for _, row in df.iterrows()
    ]
)

y = df["eval"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.uint8)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.uint8)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)

In [ ]:
model = keras.Sequential(
    [
        layers.Input(shape=(6, 64)),
        layers.Dense(64, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Flatten(),
        layers.Dense(1),
    ]
)

model.compile(optimizer="adam", loss="mean_squared_error")

model.fit(X_train, y_train, epochs=100, batch_size=64)

model.evaluate(X_test, y_test)

Epoch 1/100
601/601 [==============================] - 4s 5ms/step - loss: 1824560.5000
Epoch 2/100
601/601 [==============================] - 4s 6ms/step - loss: 1797858.1250
Epoch 3/100
601/601 [==============================] - 4s 6ms/step - loss: 1737351.2500
Epoch 4/100
601/601 [==============================] - 4s 6ms/step - loss: 1697889.2500
Epoch 5/100
601/601 [==============================] - 4s 6ms/step - loss: 1677426.8750
Epoch 6/100
601/601 [==============================] - 4s 6ms/step - loss: 1664814.1250
Epoch 7/100
601/601 [==============================] - 4s 6ms/step - loss: 1655480.2500
Epoch 8/100
601/601 [==============================] - 4s 6ms/step - loss: 1647683.7500
Epoch 9/100
601/601 [==============================] - 4s 6ms/step - loss: 1640972.2500
Epoch 10/100
601/601 [==============================] - 4s 6ms/step - loss: 1634869.7500
Epoch 11/100
601/601 [==============================] - 4s 7ms/step - loss: 1629107.8750
Epoch 12/100
601/601 [========

1615302.0